# Module gmso.abc
This module provides the abstract base classes for all other core data structures used in gmso. Our abstract base classes inherit from [pydantic](https://pydantic-docs.helpmanual.io/)'s `BaseModel` class which provides type hints as well as runtime data validation together with out-of-the-box serialization. The module structure is as follows:
```
gmso/abc 
├── abstract_connection.py 
├── abstract_potential.py 
├── abstract_site.py 
├── gmso_base.py 
```


1. [`gmso_base.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/abc/gmso_base.py): Defines the class `GMSOBase` i.e. The base class for all our other classes that tweaks pydantic's `BaseModel` class to provide an `id`-based hasing as well as injects numpydoc style docstrings from the fields of the class.
---
2. [`abstract_site.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/abc/abstract_site.py): Defines the `Site` class which provides a basic topology site with following attributes: (a.) name (b.) position (c.) label
---
3. [`abstract_potential.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/abc/abstract_potential.py): Defines the abstract `Potential` class which is the base class for our `ParametricPotentials` as well as `PotentialTemplates`.
---
4. [`abstract_connection.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/abc/abstract_site.py): Defines the abstract `Connection` class which is the base class for our `Bond`, `Angle`, `Dihedral` and `Improper` classes.


## Extensibility 
The implementation of abstract base classes allows for a consistent and systematic expansion of the library. The `gmso.abc` would allow new data classes to be implemented as needs arise with built-in common attributes/methods and type-check. 

## Example: Implementing a Bead

The `Bead` class can now be implemented as a subclass of the abstract `Site` class. We can use the existing attributes from the super class like `name`, `position` etc... and define new attributes and methods for `Bead`. The goal is the consolidation of as many universal characteristics of a generic topology site into a base class (`Site`) and tweak its down-stream usage according to the needs of a particular site (like an `Atom` or a `Bead`). Usage of `Site` to create a `Bead` class is shown below:

In [ ]:
import warnings
warnings.simplefilter('ignore')
import unyt as u
from pydantic import Field, ValidationError

from gmso.abc.abstract_site import Site


class Bead(Site):
    __base_doc__ = "Basic Bead class inheriting from the Site Class"
    mass_: u.unyt_quantity = Field(
        default=1.0*u.amu,
        description='Mass of the bead'
    )
        
    charge_: u.unyt_quantity = Field(
        default=0.0*u.elementary_charge,
        description='Charge of the bead'
    )
    
    class Config:
        fields = {
            'mass_': 'mass',
            'charge_': 'charge'
        }
        alias_to_fields = {
            'mass': 'mass_',
            'charge': 'charge_'
        }
    
my_bead = Bead()
my_bead.name  # When you inherit, the attribute(field) `name` is injected as the class name(Bead in this case)

# We use pydantic for validation as well, for example if you assign a string to charge by accident :)
try:
    my_bead.charge = 'Some weird charge string'
except ValidationError as e:
    print(e)

In [ ]:
# Documentation is injected automatically as well
%pdoc Bead